## Char-RNN (prediction)

This notebook describes the use of a char-RNN.

The notebook has been apoted from: https://github.com/mineshmathew/char_rnn_karpathy_keras/blob/master/char_rnn_of_karpathy_keras.py

The training is described in [15_char_rnn_didactic_no_state_training.ipynb](https://github.com/tensorchiefs/dl_course/blob/master/notebooks/15_char_rnn_didactic_no_state_training.ipynb).

In [1]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM,SimpleRNN
from keras.layers.wrappers import TimeDistributed
from keras.utils.data_utils import get_file
import numpy as np
from time import sleep
import random
import sys
import pickle

Using TensorFlow backend.


In [2]:
char_indices, indices_char = pickle.load( open( "15_chars.p", "rb" ) )

In [3]:
len(char_indices)

106

In [4]:
# Loading the model
import urllib
import os
if not os.path.isfile('Didactic_no_state'):
    urllib.request.urlretrieve("https://www.dropbox.com/s/ibs345wdjtxuy1t/Didactic_no_state?dl=1","Didactic_no_state")
%ls -hl Didactic_no_state

-rw-r--r-- 1 root root 27M Mar 23 14:24 Didactic_no_state


In [5]:
model = keras.models.load_model('Didactic_no_state')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 40, 512)           1267712   
_________________________________________________________________
lstm_2 (LSTM)                (None, 40, 512)           2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 512)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 40, 106)           54378     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 40, 106)           0         
Total params: 3,421,290.0
Trainable params: 3,421,290.0
Non-trainable params: 0.0
_________________________________________________________________


In [ ]:
def _sample(a, temperature):
    # helper function to sample an index from a probability array
    logit = np.log(a) 
    p = np.exp(logit / temperature)
    #1.001 to be on the save side, sum(p) < 1 for np.random.multinomial
    p /= (1.001 * np.sum(p))
    return np.argmax(np.random.multinomial(1, p, 1))

def print_sample(sentence = 'Zur Didaktik des integrativen Unterricht', temp=0.5):
    generated = ''
    start_index = 0
    maxlen = 40
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x = np.zeros((1, maxlen, len(char_indices)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = (model.predict_proba(x, verbose=0))[0,-1,:]
        #print(preds)
        next_index = _sample(preds, temp)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [ ]:
print("---- Before Training ----- ")
print_sample()

---- Before Training ----- 
----- Generating with seed: "Zur Didaktik des integrativen Unterricht"
Zur Didaktik des integrativen Unterrichts
Eine gewisse Gefahr erkennen die befragte

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


n Lehrpersonen auch ganze 

#### Some samples 

We started with "Zur Didaktik des integrativen Unterricht" and temp=1/2

```
Zur Didaktik des integrativen Unterrichts
„Das bedeutet auch die Begriffe Heterogenität und deine Vorstellung der Didaktik im integrativen Unter

Zur Didaktik des integrativen Unterrichts
Frontalunterricht miteinander oder wir haben auch Kreisgespräche innerhalb einer Unterrichtseinheit eingesetzt werden

Zur Didaktik des integrativen Unterrichts
Die Sachbegegnung soll dabei nicht nur auf Hören und Sehen beschränkt werden, „sondern in der Motorik, im handelnden Unterrichtsstil gefunden hat, der in optivieren der Ergebnisse bewusst ist.

```